In [1]:
import numpy as np
import pandas as pd
import pyarrow
import fastparquet
from pandasql import sqldf
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import string

/Users/isabellajohansson/miniconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<h1><font color='green'>1. Segment the users into segments that represent the ways that a users is paid. For instance bi-weekly vs pension vs gig economy. Ensure that you can model a user with multiple income streams. Please provide an example segmentation
    </font></h1>

In [2]:
#Data pre processing
transactions = pd.read_parquet("data_science_take_home_transactions.parquet", engine='pyarrow')
advances = pd.read_parquet("data_science_take_home_advances.parquet", engine='pyarrow')

In [3]:
###1. Convert 'account_id' to numerical values
# Create a copy of the 'account_id' column as 'account_id_string'
transactions['account_id_string'] = transactions['account_id'].copy()
advances['account_id_string'] = advances['account_id'].copy()
# Create a label encoder object
label_encoder = LabelEncoder()
# Transform 'account_id' column to numerical values and replace the original column 
transactions['account_id'] = label_encoder.fit_transform(transactions['account_id'])
advances['account_id'] = label_encoder.fit_transform(advances['account_id'])

###2. Only looking at postivie numbers and ignoring charges (for now?).
#Noticed that for payment_channel: "online" and "in-store" can be filtered out bc they seem to exclude salaries.
selected_data = transactions[(transactions['payment_channel'] != 'online') &
                             (transactions['payment_channel'] != 'in store') &
                             (transactions['f0_'] > 0)]
###3. Lets ignore irrelevant columns: 

columns_to_remove = ['version', 'currency_code', 'pending', 'removed', 'selected_data', 'city', 'region', 'payment_meta_payment_processor',
                    'authorized_date', 'payment_meta_payment_method', 'payment_channel']
cleaned_data = selected_data.drop(columns=columns_to_remove, axis=1, errors='ignore')
#data_cleaned

### Example of segmentation:
#### 1. Transaction Type Identification: Categorize transactions that represent income (like direct deposits, salary payments, pension deposits, gig economy)payments, etc.). You might use keywords, transaction types, or specific account references to identify these transactions.

In [4]:
# Define keywords or phrases that indicate different income sources -- 
### NEED TO ADD A LOT MORE KEY-WORDS HERE..this takes a lot of manual work

salary_keywords = ['salary', 'paycheck', 'wage', 'payroll']
pension_keywords = ['pension', 'retirement']
gig_keywords = ['gig', 'lyft', 'uber', 'zelle']
non_income_keywords = ['transfer', 'visa', 'atm','transfer']

def categorize_transaction(row):
    name = row['name'].lower()
    if any(keyword in name for keyword in salary_keywords):
        return 'Salary'
    elif any(keyword in name for keyword in pension_keywords):
        return 'Pension'
    elif any(keyword in name for keyword in gig_keywords):
        return 'Gig Economy'
    elif any(keyword in name for keyword in non_income_keywords):  # Check for 'Non-Income' keywords
        return 'Non-Income'  # Assign 'Non-Income' category for transactions with specified keywords
    else:
        return 'Non-Income'  # Default to 'Non-Income' for other non-matching transactions

# Apply categorization function to create a new 'transaction_category' column
cleaned_data['transaction_category'] = cleaned_data.apply(categorize_transaction, axis=1)

# Display the segmented data
cleaned_data[50:100]

,account_id,id,f0_,date,name,bank_account_id,account_id_string,transaction_category
274,355,pnonAM1Vw4iKPMvLEywmf5pXN8ONoXHZoLJyj,57.00,2022-12-27,Venm,0B6Ba0okwDu7ZLxjzKDYFJDbymvjxzFxrQAnd,5tq7bxhrwhkg0,Non-Income
277,355,X474mEZoY8SB3JeV8oL5ivOQqqQnDqI0RExrQ,24.63,2021-11-26,Venm,0B6Ba0okwDu7ZLxjzKDYFJDbymvjxzFxrQAnd,5tq7bxhrwhkg0,Non-Income
280,355,O3Z3qxv4YOt1dk3J5bDyILnb6axYX6tVryX1P,5.00,2022-11-26,Deposit ZEL*MADELYN HERNANDEZ / Visa Direct AZ...,0B6Ba0okwDu7ZLxjzKDYFJDbymvjxzFxrQAnd,5tq7bxhrwhkg0,Non-Income
281,355,JD7D6034gJTpdqDjrzJ5FrJbQbNnvLCXjyEPE,100.00,2022-07-17,Deposit ZEL*MARCUS GENDALE Visa / Direct AZ Da...,0B6Ba0okwDu7ZLxjzKDYFJDbymvjxzFxrQAnd,5tq7bxhrwhkg0,Non-Income
289,355,Rk7kyZrKYJskLpV5MOonHgRK66KJm6H4NwnjY,3001.32,2022-02-28,Deposit FULLERTON ELEM / TYPE: DEPOSIT-22 ID: ...,0B6Ba0okwDu7ZLxjzKDYFJDbymvjxzFxrQAnd,5tq7bxhrwhkg0,Non-Income
294,355,yQ5QOMvEy6u3EMZwn0y4UbD1d1qPBkU3mQA8e,0.11,2022-04-11,SchoolsFirst FC,0B6Ba0okwDu7ZLxjzKDYFJDbymvjxzFxrQAnd,5tq7bxhrwhkg0,Non-Income
308,355,X474mEZoY8SB3JeV8oL5ivOQLQjRpVC0RExJa,436.00,2022-06-22,Venm,0B6Ba0okwDu7ZLxjzKDYFJDbymvjxzFxrQAnd,5tq7bxhrwhkg0,Non-Income
313,355,3K1KQ47NDquDdPZonQw3FLjXOX14yVH7a1wkb,0.14,2022-04-11,SchoolsFirst FC,0B6Ba0okwDu7ZLxjzKDYFJDbymvjxzFxrQAnd,5tq7bxhrwhkg0,Non-Income
325,764,Pepy3ELkKXugk6rNV6MwU067QYzXAxtJQ0Jmq,820.25,2022-03-17,NOI HAWAII NET PAYROLL MICHELLE L MCKENZ,0JXBQ6qvngHDJPVX7PwZt0dmqYnD3NIxke0MQ,cwx5ffn5zcszg,Salary
327,764,5bXAJqkan6Cx56VXj6P9feB1p9wPZDCV9jVBV,831.87,2021-12-09,NOI HAWAII NET PAYROLL MICHELLE L MCKENZ,0JXBQ6qvngHDJPVX7PwZt0dmqYnD3NIxke0MQ,cwx5ffn5zcszg,Salary


In [5]:
# Assuming 'category' is the column where you have stored the categories
category_counts = cleaned_data['transaction_category'].value_counts()

# Displaying the counts of different categories
print(category_counts)

transaction_category
Non-Income     306946
Gig Economy     32281
Salary          14206
Pension            49
Name: count, dtype: int64


In [6]:
cleaned_data['date'] = pd.to_datetime(cleaned_data['date'])

# Group transactions by 'name' and calculate the frequency of occurrences per month
name_frequency = cleaned_data.groupby('name').resample('M', on='date').size().reset_index(name='monthly_count')

# Filter transactions occurring once a month or bi-weekly
monthly_transactions = name_frequency[name_frequency['monthly_count'] >= 1]  # Transactions occurring monthly or more
biweekly_transactions = name_frequency[name_frequency['monthly_count'] >= 2]  # Transactions occurring bi-weekly or more

# Display transactions occurring once a month
print("Transactions occurring once a month or more:")
print(monthly_transactions)

# Display transactions occurring bi-weekly
print("\nTransactions occurring bi-weekly or more:")
print(biweekly_transactions)

Transactions occurring once a month or more:
                                                     name       date  \
0                                         # 182 Howard St 2022-07-31   
1          # 182 Howard St. San Francisco CA Card 15 #407 2022-10-31   
2                # APPLE CASH Visa Direct CA Card 15 #407 2022-03-31   
4                # APPLE CASH Visa Direct CA Card 15 #407 2022-05-31   
5                # APPLE CASH Visa Direct CA Card 15 #994 2022-04-30   
...                                                   ...        ...   
223580  w8hWoEwIJg23PR 12/27 #XXXXX5908 PMNT RCVD Door... 2021-12-31   
223581  w8hWoEwIJg23PR 12/28 #XXXXX1798 PMNT RCVD Door... 2021-12-31   
223582  w8hWoEwIJg23PR 12/29 #XXXXX9016 PMNT RCVD Door... 2021-12-31   
223583                  www.Paywithfour.com, Httpswww.Pay 2022-07-31   
223584  zFunds DES:XXXXX40601 ID:762863 INDN:Jessica S... 2023-02-28   

        monthly_count  
0                   1  
1                   2  
2                 

#### 2. Frequency: Segmentation by dates involves identifying patterns in transactional dates to categorize them as 'bi-weekly salary', 'monthly pension', or other frequencies.

In [7]:
# Assuming 'date' column is in datetime format, if not, convert it using pd.to_datetime()
cleaned_data['date'] = pd.to_datetime(cleaned_data['date'])

# Func ttion to categorize frequency based on date
def categorize_frequency(row):
    if row['transaction_category'] == 'Salary':
        # Assuming bi-weekly salary payments every 14 days
        if row['date'].day % 14 == 0:
            return 'Bi-Weekly Salary'
        # Add more conditions for other salary frequencies if needed
        else:
            return 'Other Salary Frequency'
    elif row['transaction_category'] == 'Pension':
        # Assuming monthly pension payments
        if row['date'].day == 1:
            return 'Monthly Pension'
        # Add more conditions for other pension frequencies if needed
        else:
            return 'Other Pension Frequency'
    # Add similar conditions for other categories if needed
    else:
        return 'Non-Income'

# Apply categorization function to create a new 'frequency_category' column
cleaned_data['frequency_category'] = cleaned_data.apply(categorize_frequency, axis=1)

# Display the segmented data based on frequency
print(cleaned_data[['transaction_category', 'frequency_category']])


        transaction_category frequency_category
4                 Non-Income         Non-Income
11                Non-Income         Non-Income
14                Non-Income         Non-Income
16                Non-Income         Non-Income
18                Non-Income         Non-Income
...                      ...                ...
1962474           Non-Income         Non-Income
1962475           Non-Income         Non-Income
1962476           Non-Income         Non-Income
1962477           Non-Income         Non-Income
1962478           Non-Income         Non-Income

[353482 rows x 2 columns]


In [8]:
#check:SELECT transaction_category, frequency_category FROM cleaned_data WHERE transaction_category != frequency_category


result = cleaned_data[cleaned_data['transaction_category'] != cleaned_data['frequency_category']][['transaction_category', 'frequency_category']]

# Displaying the selected columns from the filtered data
print(result)

        transaction_category      frequency_category
145              Gig Economy              Non-Income
325                   Salary  Other Salary Frequency
327                   Salary  Other Salary Frequency
333                   Salary  Other Salary Frequency
334                   Salary  Other Salary Frequency
...                      ...                     ...
1961640          Gig Economy              Non-Income
1961642          Gig Economy              Non-Income
1961856          Gig Economy              Non-Income
1961857          Gig Economy              Non-Income
1961858          Gig Economy              Non-Income

[46536 rows x 2 columns]


<h1><font color='green'>2. How would you calculate the next pay day for each user? How would you test the performance of model? Please provide your answer as a set of rules or a proposed machine learning model (ensemble will also work).</font></h1>

#### To predict next pay date for each user I would probaly need to experiment with multiple algorithms..for a rule based approch:

- Pattern Recognition: Analyze past transactional data to identify regular intervals between income transactions for each user.

- User-Specific Patterns: Account for individual variations like irregular income schedules or multiple income sources.

- Domain Knowledge Rules: Incorporate known information such as typical payroll schedules or known payment dates for specific income sources.


<h1><font color='green'>3. Do these segments have any predictive power in determining the likelihood that a user will repay us on time?</font></h1>


#### Yes, for example through 'income stability' such as fixed bi-weekly/monthy income may exhibit better repayment behavior due to stable cash flow, making them more likely to repay on time. Whereas irregular income can lead to higher uncertaint in repayment timing.Using logistic regression or descion tree we can assess repayment likelyhood...

<h1><font color='green'>4. How could we use nlp to improve the segmentation results:
- What techniques make sense to use? 
- How will you go about building a model?</font></h1>


#### We can use nlp to extract more deails and nuanced information from the trancations description 'name'. In terms of teqniques this would be by using 'word embeddins' such as Word2Vec or GloVe which can capture semantic relationships between words in transaction descriptions. This helps in identifying similar words and understanding context better. (Or DLM)

#### To build a model I would:
- 1. Define objective: outline the above 
- 2. Gather data and ensure its well labelled
- 3. Process data: remove noise etc.)
- 4. Exloratory analysis: Understand the characteristics of the dataset, check class distributions, word frequencies, and analyze patterns in the text.
- 5. Convert text into numerical or vector representations (TF-IDF, word embeddings like Word2Vec, GloVe, or contextual embeddings like BERT).
- 6. Select model: Choose an appropriate model based on the task and the dataset size.
- 7. Model training: Train the selected model on the preprocessed data. Use a portion of the dataset for training and validation.
- 8. Evaluation: Analyze model errors, reiterate steps, and experiment with different architectures, feature representations, or preprocessing techniques to improve performance.
- 9. Deploy model

<h1><font color='green'> 5 - What evaluation metrics will you use to asses the performance of the model? - if you are given more time and data, how will you use it to make a better model? Be as specific as possible, highlighting what data you will need and if you would be able to do something more sophisticated, given more time.</font></h1>

#### I would look at accuracy, precision, recall,m f1-score.. and evaluating the trade-off between precision and recall. I might also use BLEU (Bilingual Evaluation Understudy): Measures how well a language model predicts a sample.

#### Given more time and data, a more sophisticated approach would involve a combination of leveraging additional data, experimenting with state-of-the-art architectures, and fine-tuning models and parameters to achieve higher performance and robustness in handling diverse NLP tasks. With additional data (especially if it covers diverse scenarios or specific edge cases) I can improve the model's robustness and generalization. Whereas more time allows for extensive hyperparameter tuning to find the optimal settings for the model. 

<h1><font color='green'>	6.	As a lender we want to optimize our cash flows, while still providing a good product experience for the user. We would like to be repaid within 35 days of the due date and for the user the to be able to hold an advance for at least 7 days before needing to repay. We also don't want users to have due dates prior to their next "pay day". Finally we want the repayment dates to be easily understood by our team and users. Given what you've learned from the dataset what would you recommend as an algorithm for setting repayment dates for users and why?</font></h1>

#### Proposed Algorithm:
Payday Alignment: Identify the user's income pattern (e.g., bi-weekly, monthly) based on their transaction history. Ensure that the due date aligns with their income schedule, avoiding due dates prior to their next payday.

Advance Holding Period: Allow users to hold an advance for at least 7 days before requiring repayment. Set the due date accordingly to provide users with this grace period.

Optimal Repayment Period: Aim to receive repayments within 35 days of the due date. Calculate the due date considering this optimal repayment window from the initial due date.

Clear Communication: Ensure the due dates are easily understandable by both users and your team. Use simple and clear explanations or labels for the due dates to minimize confusion.

Benefits of the Algorithm:
Cash Flow Optimization: Aligning due dates with user income patterns can improve repayment rates, optimizing cash flows for the lending institution.
User Experience: Providing a grace period and aligning due dates with income schedules enhances the user experience, reducing financial stress and likelihood of defaults.
Transparency: Clear and easily understandable due dates improve communication and reduce confusion for both users and the lending team.
Algorithm Selection Justification:
A hybrid approach combining rule-based criteria with data-driven insights from user transaction histories and predictive modeling offers a customized and adaptive solution. This approach optimizes due dates considering income patterns, financial feasibility, and user behavior while aligning with the lender's cash flow requirements. Moreover, it allows for flexibility to adapt to individual user circumstances and provides a transparent and user-friendly repayment schedule.